In [ ]:
%reload_ext autoreload
%autoreload 2


import sys, os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np

from tqdm.notebook import tqdm

from xgboost import XGBRegressor, XGBClassifier
from econml.dml import NonParamDML, LinearDML
from econml.dr import LinearDRLearner, ForestDRLearner
from econml.metalearners import XLearner
from econml import metalearners

import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR

from src.data.data_module import generate_data_exp, _generate_covariates
from src.data.utils import split_eval_cate

In [ ]:
def get_regressor(missing_value):
    return XGBRegressor(missing=missing_value, eval_metric="logloss")


def get_classifier(missing_value):
    return XGBClassifier(missing=missing_value, eval_metric="logloss")


def get_imputer(missing_value):
    return IterativeImputer(
        max_iter=1500, tol=15e-4, random_state=None, missing_values=missing_value
    )


learners = {
    "T": lambda missing_value: metalearners.TLearner(
        models=get_regressor(missing_value)
    ),
    "X": lambda missing_value: XLearner(
        models=get_regressor(missing_value),
        propensity_model=get_classifier(missing_value),
        cate_models=get_regressor(missing_value),
    ),
    "DR": lambda missing_value: ForestDRLearner(
        model_propensity=get_classifier(missing_value),
        model_regression=get_regressor(missing_value),
    ),
}


def evaluate(ground_truth, estimate, W):
    PEHE = np.sqrt(((estimate - ground_truth) ** 2).mean())
    PEHE_0 = np.sqrt(((estimate[W == 0] - ground_truth[W == 0]) ** 2).mean())
    PEHE_1 = np.sqrt(((estimate[W == 1] - ground_truth[W == 1]) ** 2).mean())
    return PEHE, PEHE_0, PEHE_1

In [ ]:
# EXP. SETTINGS
runs = 100
sims = 10

assert runs % sims == 0


train_size = 5000

d = 20
z_d_dim = 10
amount_of_missingness = 0.3
missing_value = -1

learner = "DR"


# DEBUG SETTINGS
verbose = False

In [ ]:
ground_truth_cate = []
ground_truth_cate_0 = []
ground_truth_cate_1 = []


PEHE_impute_all = []
PEHE_impute_nothing = []
PEHE_impute_smartly = []
PEHE_impute_wrongly = []

effect_impute_all = []
effect_impute_nothing = []
effect_impute_smartly = []
effect_impute_wrongly = []

effect_impute_all_0 = []
effect_impute_nothing_0 = []
effect_impute_smartly_0 = []
effect_impute_wrongly_0 = []

effect_impute_all_1 = []
effect_impute_nothing_1 = []
effect_impute_smartly_1 = []
effect_impute_wrongly_1 = []
for _ in tqdm(range(sims)):
    X, X_, Y0, Y1, Y, CATE, W, Z_up, Z_down = generate_data_exp(
        train_size * 2, d, z_d_dim, amount_of_missingness, missing_value=missing_value
    )

    assert 10 < train_size < len(X)

    for _ in tqdm(range(int(runs / sims)), leave=False):
        idxs = np.random.choice(range(len(X)), size=train_size, replace=False)
        include_idx = set(idxs)
        mask = np.array([(i in include_idx) for i in range(len(X))])

        X_train, Y_train, W_train, CATE_train = X_[mask], Y[mask], W[mask], CATE[mask]
        X_test, Y_test, W_test, CATE_test = X_[~mask], Y[~mask], W[~mask], CATE[~mask]

        ground_truth_cate.append(CATE_test)
        ground_truth_cate_0.append(CATE_test[W_test == 0])
        ground_truth_cate_1.append(CATE_test[W_test == 1])

        # IMPUTE ALL
        imputer = get_imputer(missing_value)
        imputer.fit(X_train)
        X_train_preprocessed = imputer.transform(X_train)
        X_test_preprocessed = imputer.transform(X_test)

        est_impute_all = learners[learner](missing_value)
        est_impute_all.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_all.append(evaluate(CATE_test, te, W_test))
        effect_impute_all.append(est_impute_all.effect(X_test_preprocessed))
        effect_impute_all_0.append(
            est_impute_all.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_all_1.append(
            est_impute_all.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("all", X_train.min())

        # IMPUTE NOTHING
        treatment_effects_impute_nothing = []
        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        est_impute_nothing = learners[learner](missing_value)
        est_impute_nothing.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_nothing.append(evaluate(CATE_test, te, W_test))
        effect_impute_nothing.append(est_impute_nothing.effect(X_test_preprocessed))
        effect_impute_nothing_0.append(
            est_impute_nothing.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_nothing_1.append(
            est_impute_nothing.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("nothing", X_train.min())

        # IMPUTE SMARTLY
        treatment_effects_impute_smartly = []
        imputer_smart = get_imputer(missing_value)
        imputer_smart.fit(X_train[:, z_d_dim:])

        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        X_train_preprocessed[:, z_d_dim:] = imputer_smart.transform(
            X_train[:, z_d_dim:]
        )
        X_test_preprocessed[:, z_d_dim:] = imputer_smart.transform(X_test[:, z_d_dim:])

        est_impute_smartly = learners[learner](missing_value)
        est_impute_smartly.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_smartly.append(evaluate(CATE_test, te, W_test))
        effect_impute_smartly.append(est_impute_smartly.effect(X_test_preprocessed))
        effect_impute_smartly_0.append(
            est_impute_smartly.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_smartly_1.append(
            est_impute_smartly.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("smart down", X_train[:, :z_d_dim].min())
            print("smart up", X_train[:, z_d_dim:].min())

        # IMPUTE WRONGLY
        treatment_effects_impute_wrongly = []
        imputer_wrongly = get_imputer(missing_value)
        imputer_wrongly.fit(X_train[:, :z_d_dim])

        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        X_train_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(
            X_train[:, :z_d_dim]
        )
        X_test_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(
            X_test[:, :z_d_dim]
        )

        est_impute_wrongly = learners[learner](missing_value)
        est_impute_wrongly.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_wrongly.append(evaluate(CATE_test, te, W_test))
        effect_impute_wrongly.append(est_impute_wrongly.effect(X_test_preprocessed))
        effect_impute_wrongly_0.append(
            est_impute_wrongly.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_wrongly_1.append(
            est_impute_wrongly.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("wrong down", X_train[:, :z_d_dim].min())
            print("wrong up", X_train[:, z_d_dim:].min())

In [ ]:
all_means, all_stds = split_eval_cate(effect_impute_all, ground_truth_cate, sims)
no_means, no_stds = split_eval_cate(effect_impute_nothing, ground_truth_cate, sims)
smart_means, smart_stds = split_eval_cate(
    effect_impute_smartly, ground_truth_cate, sims
)
wrong_means, wrong_stds = split_eval_cate(
    effect_impute_wrongly, ground_truth_cate, sims
)

all_means_0, all_stds_0 = split_eval_cate(
    effect_impute_all_0, ground_truth_cate_0, sims
)
no_means_0, no_stds_0 = split_eval_cate(
    effect_impute_nothing_0, ground_truth_cate_0, sims
)
smart_means_0, smart_stds_0 = split_eval_cate(
    effect_impute_smartly_0, ground_truth_cate_0, sims
)
wrong_means_0, wrong_stds_0 = split_eval_cate(
    effect_impute_wrongly_0, ground_truth_cate_0, sims
)

all_means_1, all_stds_1 = split_eval_cate(
    effect_impute_all_1, ground_truth_cate_1, sims
)
no_means_1, no_stds_1 = split_eval_cate(
    effect_impute_nothing_1, ground_truth_cate_1, sims
)
smart_means_1, smart_stds_1 = split_eval_cate(
    effect_impute_smartly_1, ground_truth_cate_1, sims
)
wrong_means_1, wrong_stds_1 = split_eval_cate(
    effect_impute_wrongly_1, ground_truth_cate_1, sims
)

print("# SETUP")
print(f"# amount_of_missingness = {amount_of_missingness}")

print(f"#   ALL IMPUTATION  :\t{np.mean(all_means)}\t{np.mean(all_stds)}")
print(f"#   NO IMPUTATION   :\t{np.mean(no_means)}\t{np.mean(no_stds)}")
print(f"#   SMART IMPUTATION:\t{np.mean(smart_means)}\t{np.mean(smart_stds)}")
print(f"#   WRONG IMPUTATION:\t{np.mean(wrong_means)}\t{np.mean(wrong_stds)}")
print(f"#   ---------------- PEHE_0")
print(f"#   ALL IMPUTATION  :\t{np.mean(all_means_0)}\t{np.mean(all_stds_0)}")
print(f"#   NO IMPUTATION   :\t{np.mean(no_means_0)}\t{np.mean(no_stds_0)}")
print(f"#   SMART IMPUTATION:\t{np.mean(smart_means_0)}\t{np.mean(smart_stds_0)}")
print(f"#   WRONG IMPUTATION:\t{np.mean(wrong_means_0)}\t{np.mean(wrong_stds_0)}")
print(f"#   ---------------- PEHE_1")
print(f"#   ALL IMPUTATION  :\t{np.mean(all_means_1)}\t{np.mean(all_stds_1)}")
print(f"#   NO IMPUTATION   :\t{np.mean(no_means_1)}\t{np.mean(no_stds_1)}")
print(f"#   SMART IMPUTATION:\t{np.mean(smart_means_1)}\t{np.mean(smart_stds_1)}")
print(f"#   WRONG IMPUTATION:\t{np.mean(wrong_means_1)}\t{np.mean(wrong_stds_1)}")

In [ ]:
# RESULTS (temp, see below)

# V
# T-Learner (100 runs | n=5000:5000 | XGBoost)
# SETUP
# amount_of_missingness = 0.3
#   ALL IMPUTATION  :	0.7603202228807558	0.05107655476272003
#   NO IMPUTATION   :	0.6906018297012612	0.07293375356277434
#   SMART IMPUTATION:	0.4605443542126303	0.045559141435647814
#   WRONG IMPUTATION:	0.9158624744688488	0.06479239894963464
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	0.7371179935236377	0.0813447454038048
#   NO IMPUTATION   :	0.7015184811570604	0.10091128810622667
#   SMART IMPUTATION:	0.572054086285261	0.07933149623910837
#   WRONG IMPUTATION:	0.9351743987244323	0.12057528790964936
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	0.7726199464001329	0.05520143480388414
#   NO IMPUTATION   :	0.6881436442651043	0.0901607628080812
#   SMART IMPUTATION:	0.40971083141461895	0.045350667352497306
#   WRONG IMPUTATION:	0.9183522971586949	0.07513038656333801


# V
# DR-Learner (30 runs | n=5000:5000 | XGBoost)
# SETUP
# amount_of_missingness = 0.3
#   ALL IMPUTATION  :	1.3674804785864336	1.7318280165796296
#   NO IMPUTATION   :	0.9409536956704727	1.9431432104562432
#   SMART IMPUTATION:	0.20421566866874663	0.22411214110370406
#   WRONG IMPUTATION:	4.365789991043024	8.82361783733213
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	1.2083191574779995	1.6106075455876951
#   NO IMPUTATION   :	0.8130941858636677	1.2876995138351313
#   SMART IMPUTATION:	0.1787563949407937	0.20287619043864713
#   WRONG IMPUTATION:	4.2306427399228905	9.056379962065844
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	1.4419080960316477	1.8027164353381573
#   NO IMPUTATION   :	0.997339557310883	2.3179472556207465
#   SMART IMPUTATION:	0.216936220213687	0.23653557233533617
#   WRONG IMPUTATION:	4.432220414868022	8.749143751110289


# V
# X-Learner (100 runs | n=5000:5000 | XGBoost)
# SETUP
# amount_of_missingness = 0.3
#   ALL IMPUTATION  :	0.6149946402626558	0.06390551930603257
#   NO IMPUTATION   :	0.3027909087842893	0.08597057400122958
#   SMART IMPUTATION:	0.2116881614969457	0.03239196614270469
#   WRONG IMPUTATION:	0.4912969947275013	0.10914276845610779
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	0.6272957845922045	0.07083868937874815
#   NO IMPUTATION   :	0.2907498976394652	0.10774536989843311
#   SMART IMPUTATION:	0.2556986684662631	0.06254388476455
#   WRONG IMPUTATION:	0.5198024390023097	0.15692764578598367
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	0.6097365193131001	0.06839144126845112
#   NO IMPUTATION   :	0.3091613781100213	0.09853250057067761
#   SMART IMPUTATION:	0.19157833135487273	0.03664993710259606
#   WRONG IMPUTATION:	0.4803861176358356	0.13089151115646364